In [10]:
import dash 
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import numpy as np
from datetime import datetime
from dash.dependencies import *
import dash_bootstrap_components as dbc
import pandas as pd
import os
import plotly.express as px
import datetime
from statistics import *
from dash import dash_table

In [14]:
#本步是读入可视化页面中‘基金产品图’所需要的数据
weekly_data = pd.read_excel('output-周度.xlsx')#读入周度数据
print(weekly_data)
security_name = []
for i in range(3, 23):
    security_name.append(weekly_data.columns[i])#把基金的名字存入一个列表里面方便调用
figure_name = ['净值变化折线图','动态回撤折线图']#这个列表存放在可视化平台上面的图像类型名称






     number         日期  收盘价_中证500  元图中证500指数增强1号  磐松中证500指数增强1号  \
0       140 2021-01-08  6557.5964            NaN            NaN   
1       139 2021-01-15  6417.5011            NaN            NaN   
2       138 2021-01-22  6637.8052            NaN            NaN   
3       137 2021-01-29  6346.1371            NaN            NaN   
4       136 2021-02-05  6239.8709            NaN            NaN   
..      ...        ...        ...            ...            ...   
135       5 2023-09-01  5750.8027         0.9174         1.0740   
136       4 2023-09-08  5708.6227         0.9095         1.0589   
137       3 2023-09-15  5704.6632         0.9029         1.0628   
138       2 2023-09-22  5698.8168         0.9068         1.0672   
139       1 2023-09-28  5690.7697            NaN            NaN   

     托特中证500指数增强2号  子辰盛丰指数增强三号  半鞅中证500指数增强  宽投500指数增强7号  盖亚青柯中证500指数增强1号  \
0              NaN         NaN          NaN          NaN              NaN   
1              NaN         NaN          N

In [15]:
#本步是读入基金产品表所需要的数据
table_name = ['基金净值指标','基金超额净值指标']
#接下来就开始制作具体放在可视化平台上面的表格，首先制作的是基金净值指标的表格
table_column_names = ['净值开始日期','本周收益(%)','上周收益(%)','近1月收益(%)','近3月收益(%)','近6月收益(%)','近1年收益(%)','今年以来(%)','累计收益率(%)','年化收益率(%)','年化波动率(%)','夏普比率','最大回撤(%)','卡玛比率','胜率(%)']
table_columns = {name:[] for name in table_column_names}
for i in security_name:
    security_info = weekly_data[['日期',i]]
    security_info.columns = ['日期', '基金净值']
    security_info = security_info.dropna()
    table_columns['净值开始日期'].append(security_info['日期'].iloc[0])#添加数据的起始日期
    table_columns['本周收益(%)'].append(security_info['基金净值'].iloc[-1]/security_info['基金净值'].iloc[-2]-1)
    table_columns['上周收益(%)'].append(security_info['基金净值'].iloc[-2]/security_info['基金净值'].iloc[-3]-1)
    table_columns['近1月收益(%)'].append(security_info['基金净值'].iloc[-1]/security_info['基金净值'].iloc[-4]-1)
    table_columns['近3月收益(%)'].append(security_info['基金净值'].iloc[-1]/security_info['基金净值'].iloc[-12]-1)
    table_columns['近6月收益(%)'].append(security_info['基金净值'].iloc[-1]/security_info['基金净值'].iloc[-24]-1)
    if len(security_info['基金净值']) >= 52:
        table_columns['近1年收益(%)'].append(security_info['基金净值'].iloc[-1]/security_info['基金净值'].iloc[-52]-1)
    else:
        table_columns['近1年收益(%)'].append(None) 
        
    DATES = [Date.strftime('%Y-%m-%d') for Date in security_info['日期'].tolist()]
    if '2022-12-30' in DATES:
        table_columns['今年以来(%)'].append(security_info['基金净值'].iloc[-1]/security_info[security_info['日期'] == '2022-12-30']['基金净值'].values[0]-1)
    else:
        table_columns['今年以来(%)'].append(None) 
    table_columns['累计收益率(%)'].append(security_info['基金净值'].iloc[-1]/security_info['基金净值'].iloc[0]-1)
    total_interest_rate = security_info['基金净值'].iloc[-1]/security_info['基金净值'].iloc[0]-1#存放累计收益率
    total_date = len(security_info['基金净值'])
    annual_interest_rate = total_interest_rate/len(security_info['基金净值'])*52
    table_columns['年化收益率(%)'].append( annual_interest_rate)
    table_columns['年化波动率(%)'].append(stdev(security_info['基金净值'])*52**0.5)
    table_columns['夏普比率'].append((annual_interest_rate-0.03)/(stdev(security_info['基金净值'])*52**0.5))
    drawback = []
    for i in range(0, len(security_info['基金净值'])):
        max_price = max(security_info['基金净值'].iloc[0:i+1])
        drawback.append(-1*max((max_price-security_info['基金净值'].iloc[i])/max_price,0))
    table_columns['最大回撤(%)'].append(min(drawback))
    table_columns['卡玛比率'].append(annual_interest_rate/(-min(drawback)))
    chance_of_winning_count = 0
    for i in range(1, len(security_info['基金净值'])):
        if security_info['基金净值'].iloc[i] > security_info['基金净值'].iloc[i-1]:
            chance_of_winning_count += 1
    table_columns['胜率(%)'].append(chance_of_winning_count/len(security_info['基金净值']))
    
df_table_columns =  pd.DataFrame(table_columns)#将字典转换成数据框，并且在第一列加上基金名称
df_table_columns.insert(0,'基金名称', security_name)
df_table_columns[['本周收益(%)','上周收益(%)','近1月收益(%)','近3月收益(%)','近6月收益(%)','近1年收益(%)','今年以来(%)','累计收益率(%)','年化收益率(%)','年化波动率(%)','最大回撤(%)','胜率(%)']] = df_table_columns[['本周收益(%)','上周收益(%)','近1月收益(%)','近3月收益(%)','近6月收益(%)','近1年收益(%)','今年以来(%)','累计收益率(%)','年化收益率(%)','年化波动率(%)','最大回撤(%)','胜率(%)']].apply(lambda x: round(x * 100, 2))
df_table_columns[['夏普比率','卡玛比率']]=df_table_columns[['夏普比率','卡玛比率']].apply(lambda x: round(x , 2))




    
        
    
    
    
    
    
    


In [16]:
#接下来制作的是基金超额净值指标的表格
table_columns_2 = {name:[] for name in table_column_names}
for i in security_name:
    security_info = weekly_data[['日期','收盘价_中证500',i]]
    security_info.columns = ['日期','收盘价_中证500', '基金超额净值']
    security_info = security_info.dropna()
    security_info['收盘价_中证500'] = security_info['收盘价_中证500']/security_info['收盘价_中证500'].iloc[0]
    security_info['基金超额净值'] = security_info['基金超额净值']/security_info['基金超额净值'].iloc[0]
    security_info['基金超额净值'] = security_info['基金超额净值']/security_info['收盘价_中证500']
    
    table_columns_2['净值开始日期'].append(security_info['日期'].iloc[0])#添加数据的起始日期
    table_columns_2['本周收益(%)'].append(security_info['基金超额净值'].iloc[-1]/security_info['基金超额净值'].iloc[-2]-1)
    table_columns_2['上周收益(%)'].append(security_info['基金超额净值'].iloc[-2]/security_info['基金超额净值'].iloc[-3]-1)
    table_columns_2['近1月收益(%)'].append(security_info['基金超额净值'].iloc[-1]/security_info['基金超额净值'].iloc[-4]-1)
    table_columns_2['近3月收益(%)'].append(security_info['基金超额净值'].iloc[-1]/security_info['基金超额净值'].iloc[-12]-1)
    table_columns_2['近6月收益(%)'].append(security_info['基金超额净值'].iloc[-1]/security_info['基金超额净值'].iloc[-24]-1)
    if len(security_info['基金超额净值']) >= 52:
        table_columns_2['近1年收益(%)'].append(security_info['基金超额净值'].iloc[-1]/security_info['基金超额净值'].iloc[-52]-1)
    else:
        table_columns_2['近1年收益(%)'].append(None) 
        
    DATES = [Date.strftime('%Y-%m-%d') for Date in security_info['日期'].tolist()]
    if '2022-12-30' in DATES:
        table_columns_2['今年以来(%)'].append(security_info['基金超额净值'].iloc[-1]/security_info[security_info['日期'] == '2022-12-30']['基金超额净值'].values[0]-1)
    else:
        table_columns_2['今年以来(%)'].append(None) 
    table_columns_2['累计收益率(%)'].append(security_info['基金超额净值'].iloc[-1]/security_info['基金超额净值'].iloc[0]-1)
    total_interest_rate = security_info['基金超额净值'].iloc[-1]/security_info['基金超额净值'].iloc[0]-1#存放累计收益率
    total_date = len(security_info['基金超额净值'])
    annual_interest_rate = total_interest_rate/len(security_info['基金超额净值'])*52
    table_columns_2['年化收益率(%)'].append( annual_interest_rate)
    table_columns_2['年化波动率(%)'].append(stdev(security_info['基金超额净值'])*52**0.5)
    table_columns_2['夏普比率'].append((annual_interest_rate-0.03)/(stdev(security_info['基金超额净值'])*52**0.5))
    drawback = []
    for i in range(0, len(security_info['基金超额净值'])):
        max_price = max(security_info['基金超额净值'].iloc[0:i+1])
        drawback.append(-1*max((max_price-security_info['基金超额净值'].iloc[i])/max_price,0))
    table_columns_2['最大回撤(%)'].append(min(drawback))
    table_columns_2['卡玛比率'].append(annual_interest_rate/(-min(drawback)))
    chance_of_winning_count = 0
    for i in range(1, len(security_info['基金超额净值'])):
        if security_info['基金超额净值'].iloc[i] > security_info['基金超额净值'].iloc[i-1]:
            chance_of_winning_count += 1
    table_columns_2['胜率(%)'].append(chance_of_winning_count/len(security_info['基金超额净值']))
    
df_table_columns_2 =  pd.DataFrame(table_columns_2)#将字典转换成数据框，并且在第一列加上基金名称
df_table_columns_2.insert(0,'基金名称', security_name)
df_table_columns_2[['本周收益(%)','上周收益(%)','近1月收益(%)','近3月收益(%)','近6月收益(%)','近1年收益(%)','今年以来(%)','累计收益率(%)','年化收益率(%)','年化波动率(%)','最大回撤(%)','胜率(%)']] = df_table_columns_2[['本周收益(%)','上周收益(%)','近1月收益(%)','近3月收益(%)','近6月收益(%)','近1年收益(%)','今年以来(%)','累计收益率(%)','年化收益率(%)','年化波动率(%)','最大回撤(%)','胜率(%)']].apply(lambda x: round(x * 100, 2))
df_table_columns_2[['夏普比率','卡玛比率']]=df_table_columns_2[['夏普比率','卡玛比率']].apply(lambda x: round(x , 2))


    
    

In [17]:
app = dash.Dash(__name__,suppress_callback_exceptions = True)

app.layout = html.Div(children = [
    dcc.Location(id = '网址',refresh = False),
    html.Div(id = '页面选择')   
])#app页面




home_layout = html.Div(children = [
    html.H1(children = '主页面',style = {'textAlign': 'center'}),
    dcc.Link('主页面', href = '/homepage'),
    html.Br(),
    dcc.Link('基金产品研究',href = '/fundpage'),
    html.Br(),
    dcc.Link('宏观环境研究',href = '/environmentpage'),
    html.Br(),
])  #主页面，选择基金研究还是宏观环境变量研究


fund_product_layout = html.Div([
    dcc.Link('主页面', href = '/homepage'),
    html.Br(),
    html.Br(),
    dcc.Link('基金产品研究-图页面', href = '/fundpage_figure'),
    html.Br(),
    html.Br(),
    dcc.Link('基金产品研究部-表页面', href = '/fundpage_table')
])#基金产品研究页面，可以选择回到主页面，看图或者看表的页面

    
    

environment_study_layout = html.Div([
    dcc.Link('主页面', href = 'homepage'),
    html.Br(),
    html.Br(),
    html.H1(children = '宏观环境研究页面',style = {'textAlign': 'center'})
])#宏观环境研究页面，可以选择回到主页面或者其他相关操作






fund_product_layout_图页面 = html.Div([
    dcc.Link('基金产品研究',href = '/fundpage'),
    html.Br(),
    html.Br(),
    html.H1(children = '指数增强基金图页面',style = {'textAlign': 'center'}),
    html.Br(),
    html.Br(),
    html.Div(children = [
        '请选择一种图像:',
        dcc.Dropdown(id = '可选择的图像',
        options = [{'label':i, 'value':i} for i in figure_name],
                    value = '净值变化折线图')#设置下拉条选择可视化图像
    ]),
    html.Br(),
    html.Br(),
    html.Div(children = [
        '请选择一只基金:',
        dcc.Dropdown(id = '可选择的基金',
        options = [{'label':i, 'value':i} for i in security_name],
                    value =security_name[0])#设置下拉条选择基金产品
    ]),
    html.Br(),
    html.Br(),
    dcc.Graph(id = '基金图像'),#在这个位置输出基金的净值折线图或者最大回撤图
    html.Br(),
    html.Br(),
    dcc.DatePickerRange(
        id = '时间选择',
        min_date_allowed = datetime.date(2021, 1, 1),
        max_date_allowed = datetime.date(2023, 9, 28),
        start_date = datetime.date(2021, 1, 1),
        end_date = datetime.date(2023, 9, 28),
        display_format = 'YYYY-MM-DD'
    )#设置基金的开始以及结束时间
    
]) #基金产品研究的图页面，可以看图或者返回基金产品研究的页面
    
    
    
fund_product_layout_表页面 = html.Div([
    dcc.Link('基金产品研究',href = '/fundpage'),
    html.Br(),
    html.Br(),
    html.H1(children = '指数增强基金表页面',style = {'textAlign': 'center'}),
    html.Br(),
    html.Br(),
    html.Div(children = [
        '请选择一种表格:',
        dcc.Dropdown(id = '可选择的表格',
        options = [{'label':i, 'value':i} for i in table_name],
                    value = '基金净值指标')#设置下拉条选择可视化表格
    ]),
    html.Br(),
    html.Br(),
    dash_table.DataTable(
        id = '基金表格',
        fixed_rows = {'headers':True},
        fixed_columns = {'headers':True, 'data':1},
        style_table = {'minWidth':'100%','overflowX': 'auto'},
        style_cell = {'whiteSpace':'normal'},
    ),
])#基金产品研究的表页面，可以看表或者返回基金产品研究的页面
 


@app.callback(Output(component_id = '页面选择',component_property = 'children'),Input(component_id = '网址', component_property = 'pathname'))
def update_output_div(pathname):
    if pathname == '/fundpage':
        return fund_product_layout
    elif pathname == '/environmentpage':
        return environment_study_layout
    elif pathname == '/fundpage_figure':
        return fund_product_layout_图页面
    elif pathname == '/fundpage_table':
        return fund_product_layout_表页面
    else:
        return home_layout
  
    


@app.callback(Output(component_id = '基金图像',component_property = 'figure'), Input('可选择的图像','value'),Input(component_id = '可选择的基金',component_property = 'value'),Input('时间选择','start_date'),Input('时间选择','end_date'))
def update_figure(selected_figure,selected_security,start_date,end_date):   
    selected_df = weekly_data[['日期','收盘价_中证500',selected_security]]#选择我们在dropdown当中选中的基金
    selected_df = selected_df.dropna()#去除空值
    selected_df = selected_df[(selected_df['日期'] >= start_date) & (selected_df['日期'] <= end_date)]#选择我们在timerangeslider中设置的起止时间
    selected_df = selected_df.sort_values(by = '日期',ascending = True)#按照日期升序排列
    selected_df.index = list(range(selected_df.shape[0]))#将操作之后的dataframe的索引index重新变成从0开始
    selected_df['收盘价_中证500'] = selected_df['收盘价_中证500'] / selected_df['收盘价_中证500'].iloc[0]#对于中证500进行归一化
    selected_df[selected_security] = selected_df[selected_security] / selected_df[selected_security].iloc[0]#对于所选基金进行归一化
    if selected_figure == figure_name[0]:
        selected_df['超额收益'] = selected_df[selected_security]/selected_df['收盘价_中证500']#用相除的方式直接计算超额收益
        fig = px.line(selected_df, x = '日期',y = ['收盘价_中证500',selected_security,'超额收益'],labels = {'日期':'日期','value':'净值变化'},title = '净值变化折线图')#绘制折线图
        fig.update_layout(transition_duration = 500)
        return fig
    elif selected_figure == figure_name[1]:
        selected_df['超额收益'] = selected_df[selected_security]/selected_df['收盘价_中证500']#用相除的方式直接计算超额收益
        draw_back_security = []#存放基金回撤
        draw_back_standard = []#存放中证500回撤
        draw_back_difference = []#存放超额回撤
        draw_back_security.append(0)#三种回撤在开始一天都记作0，因为在第一天的时候还没有历史数据
        draw_back_standard.append(0)
        draw_back_difference.append(0)
        
        for i in range(1, len(selected_df[selected_security])):
            max_price_security = max(selected_df[selected_security][0:i])
            max_price_standard = max(selected_df['收盘价_中证500'][0:i])
            max_price_difference = max(selected_df['超额收益'][0:i])
            drawback_ratio_security = -1*max((max_price_security-selected_df[selected_security][i])/max_price_security,0)
            drawback_ratio_standard = -1*max((max_price_standard-selected_df['收盘价_中证500'][i])/max_price_standard,0)
            drawback_ratio_difference = -1*max((max_price_difference-selected_df['超额收益'][i])/max_price_difference,0)
            draw_back_security.append(drawback_ratio_security)
            draw_back_standard.append(drawback_ratio_standard)
            draw_back_difference.append(drawback_ratio_difference)
            
        selected_df[selected_security+'动态回撤'] = draw_back_security 
        selected_df['中证500动态回撤'] = draw_back_standard
        selected_df['超额回撤（几何）'] = draw_back_difference
        selected_df[selected_security+'动态回撤'] =  selected_df[selected_security+'动态回撤']*100
        selected_df['中证500动态回撤'] = selected_df['中证500动态回撤']*100
        selected_df['超额回撤（几何）'] =  selected_df['超额回撤（几何）']*100
        
        fig = px.line(selected_df, x = '日期',y = [selected_security+'动态回撤','中证500动态回撤','超额回撤（几何）'],labels = {'日期':'日期','value':'动态回撤(%)'},title = '动态回撤折线图')#绘制折线图
        fig.update_layout(transition_duration = 500)
        return fig    
    
    

@app.callback(Output('基金表格','data'),Output('基金表格','columns'),Input('可选择的表格','value'))
def update_datatable(selected_table):
    if selected_table == table_name[0]:
        data = df_table_columns.to_dict('records')
        columns = [{'name':col, 'id':col} for col in df_table_columns.columns]
    elif selected_table == table_name[1]:
        data = df_table_columns_2.to_dict('records')
        columns = [{'name':col, 'id':col} for col in df_table_columns_2.columns]
    
        
    return data, columns#这里是根据下拉菜单选择对应呈现的表格

#以下是为最终呈现的表格加上颜色
@app.callback(Output('基金表格','style_data_conditional'),Input('基金表格','data'))
def set_cell_colors(data):
    if data is None:
        return []
    style_data_conditional = []
    for dictionary in data: 
        for key, value in dictionary.items():
            if value is None:
            # 在这里将空值填充为0
                dictionary[key] = 0
    df = pd.DataFrame(data)
    
    for column in df.columns:
        if column !='index'and df[column].dtype == 'float64':#排除index列，并且只对数字类型的变量加上颜色
            max_val = max(data, key = lambda x:x[column])
            min_val = min(data, key = lambda x:x[column])
            
            for entry in data:#对于data当中的每一个数据进行循环，如果数据小于零，就令它为黄绿相间的颜色，并且越小越绿
                value = entry[column]
                if value <0:
                    color_scale = (abs(value) - 0)/(abs(min_val[column]) - 0)
                    color = f'rgb({255 - int(255*color_scale)}, 255,0)'
                elif value > 0:#如果数据大于0，就令它为红黄相间的颜色，并且越大越红
                    color_scale = (value - 0)/(max_val[column] - 0)
                    color = f'rgb(255, {255 - int(255*color_scale)}, 0)'
                else:#等于0为正黄色
                    color = 'rgb(255, 255, 0)'
                
                style_data_conditional.append({
                    'if': {'column_id': column, 'row_index': data.index(entry)},
                    'backgroundColor': color
                })
    return style_data_conditional    
         
          
        

if __name__ == '__main__':
    app.run_server(port = 2005)
    
    
    
    
    
    
    
    

In [18]:
df_table_columns.to_excel('基金净值指标.xlsx',index = False)
df_table_columns_2.to_excel('基金超额净值指标.xlsx', index = False)